In [1]:
import math
import random
#each value card has a 1:13 chance of being selected (we don't care about suits for blackjack)
#cards (value): Ace (1), 2, 3, 4, 5, 6, 7, 8, 9, 10, Jack (10), Queen (10), King (10)

def randomCard():
    card = random.randint(1,13)
    if card > 10:
        card = 10
    return card

#A hand is just a tuple e.g. (14, False), a total card value of 14 without a useable ace
#accepts a hand, if the Ace can be an 11 without busting the hand, it's useable
def useable_ace(hand):
    val, ace = hand
    return ((ace) and ((val + 10) <= 21))

def totalValue(hand):
    val, ace = hand
    if (useable_ace(hand)):
        return (val + 10)
    else:
        return val

def add_card(hand, card):
    val, ace = hand
    if (card == 1):
        ace = True
    return (val + card, ace)
#The first is first dealt a single card, this method finishes off his hand
def eval_dealer(dealer_hand):
    while (totalValue(dealer_hand) < 17):
        dealer_hand = add_card(dealer_hand, randomCard())
    return dealer_hand

#state: (player total, useable_ace), (dealer total, useable ace), game status; e.g. ((15, True), (9, False), 1)
#stay or hit => dec == 0 or 1
def play(state, dec):
    #evaluate
    player_hand = state[0] #val, useable ace
    dealer_hand = state[1]
    if dec == 0: #action = stay
        #evaluate game; dealer plays
        dealer_hand = eval_dealer(dealer_hand)

        player_tot = totalValue(player_hand)
        dealer_tot = totalValue(dealer_hand)
        status = 1
        if (dealer_tot > 21):
            status = 2 #player wins
        elif (dealer_tot == player_tot):
            status = 3 #draw
        elif (dealer_tot < player_tot):
            status = 2 #player wins
        elif (dealer_tot > player_tot):
            status = 4 #player loses

    elif dec == 1: #action = hit
        #if hit, add new card to player's hand
        player_hand = add_card(player_hand, randomCard())
        d_hand = eval_dealer(dealer_hand)
        player_tot = totalValue(player_hand)
        status = 1
        if (player_tot == 21):
            if (totalValue(d_hand) == 21):
                status = 3 #draw
            else:
                status = 2 #player wins!
        elif (player_tot > 21):
            status = 4 #player loses
        elif (player_tot < 21):
            #game still in progress
            status = 1
    state = (player_hand, dealer_hand, status)

    return state

#start a game of blackjack, returns a random initial state
def initGame():
    status = 1 #1=in progress; 2=player won; 3=draw; 4 = dealer won/player loses
    player_hand = add_card((0, False), randomCard())
    player_hand = add_card(player_hand, randomCard())
    dealer_hand = add_card((0, False), randomCard())
    #evaluate if player wins from first hand
    if totalValue(player_hand) == 21:
        if totalValue(dealer_hand) != 21:
            status = 2 #player wins after first deal!
        else:
            status = 3 #draw

    state = (player_hand, dealer_hand, status)
    return state

In [8]:
import operator
class LearningAgent():
    """An agent that learns to play blackjack."""

    def __init__(self):
        self.alpha = 0.8
        self.gamma = 0.2
        self.reset()
        self.QTable = {}
        
#         Gamma and alpha were gotten randomly, if you have a better way, do share

    def reset(self):
        self.game = initGame()

    def get_action(self):
        state = self.game[:2]
        if state not in self.QTable.keys():
            self.QTable[state] = {0: 1, 1: 1}
            return random.choice(self.QTable[state].keys())

        return max(self.QTable[state].iteritems(), key=operator.itemgetter(1))[0]

    def get_max_q(self, state):
        if state not in self.QTable.keys():
            self.QTable[state] = {0: 1, 1: 1}

        return max(self.QTable[state].values())
    
    def act(self, action):
        self.game = play(self.game, action)
        if self.game[2] == 1:
            return 0
        elif self.game[2] == 2:
            return 2
        elif self.game[2] == 3:
            return 1
        elif self.game[2] == 4:
            return -1
        
    def learn(self, currState, action, reward, nextState):
        """d: refers to the destination state"""
        maxQ = self.get_max_q(nextState)
        qReward = self.alpha * (reward - self.gamma * maxQ - self.QTable[currState][action])
        self.QTable[currState][action] += qReward

    def play(self):
        while self.game[2] == 1:
            currState = self.game[:2]
            # TODO: Select action according to your policy
            action = self.get_action()

            # Execute action and get reward
            reward = self.act(action)
            nextState = self.game[:2]

            # Update Long term reward value
            self.learn(currState, action, reward, nextState)
    
    def train(self, epochs):
        for i in xrange(epochs):
            self.play()
            self.reset()

In [14]:
epochs = 100000
agent = LearningAgent()
agent.train(epochs)

In [15]:
len(agent.QTable.keys())

1172

In [ ]:
trained_agent